<a href="https://colab.research.google.com/github/Pearl5007/AI-CAPSTONE-PROJECT/blob/main/ai_website_updated.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# =====================================================
# FINAL VERSION — FLASK + KNN + METRICS + FULL HTML + GITHUB DATA
# =====================================================

!pip install -q flask scikit-learn pandas numpy

import pandas as pd
import numpy as np
from flask import Flask, request, render_template_string
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from google.colab import output
import threading, time


# =====================================================
# LOAD DATASET FROM GITHUB
# =====================================================
url = "https://raw.githubusercontent.com/Pearl5007/AI-CAPSTONE-PROJECT/refs/heads/main/data/wisc_bc_data.csv"
df = pd.read_csv(url)
print("Dataset loaded successfully!")

# Drop ID column if exists
if "id" in df.columns:
    df = df.drop(columns=["id"])

# Fix column name mistakes if present
rename_map = {
    "points_mean": "concave_points_mean",
    "dimension_mean": "fractal_dimension_mean",
    "points_se": "concave_points_se",
    "dimension_se": "fractal_dimension_se",
    "points_worst": "concave_points_worst",
    "dimension_worst": "fractal_dimension_worst",
}
df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# Find target column automatically
target_col = None
for c in df.columns:
    if c.lower() in ("diagnosis", "target", "label"):
        target_col = c
        break
if target_col is None:
    target_col = df.columns[-1]

# Convert labels
def map_label(v):
    s = str(v).strip().upper()
    if s.startswith("M"):
        return "Malignant"
    if s.startswith("B"):
        return "Benign"
    try:
        return "Malignant" if float(s) > 0.5 else "Benign"
    except:
        return None

y = df[target_col].astype(str).map(map_label)
valid = y.isin(["Benign", "Malignant"])
df = df.loc[valid]
y = y.loc[valid]

# Features
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
numeric_cols = [c for c in numeric_cols if c != target_col]
FEATURES = numeric_cols

X = df[FEATURES].astype(float)
y = y.map(lambda v: 0 if v == "Benign" else 1)

# Scale features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.25, random_state=42, stratify=y
)

# Train model
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, y_train)

# =====================================================
# METRICS
# =====================================================
y_pred = knn.predict(X_test)

acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("\n===== MODEL METRICS =====")
print("Accuracy:", round(acc * 100, 2), "%")
print("Precision:", round(prec * 100, 2), "%")
print("Recall:", round(rec * 100, 2), "%")
print("F1 Score:", round(f1 * 100, 2), "%")
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))


# =====================================================
# FULL HTML TEMPLATE
# =====================================================

HTML = r"""
<!doctype html>
<html>
<head>
<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<title>BREAST CANCER DETECTION</title>
<style>
:root{
  --bg1: #f6fbff;
  --bg2: #f2fff9;
  --card: rgba(255,255,255,0.78);
  --glass: 12px;
  --accent1: #00c6ff;
  --accent2: #9be7ff;
  --accent3: #c8e9ff;
  --muted: #626f7a;
}

html,body{
  height:100%;margin:0;font-family:Inter, 'Segoe UI', Roboto, Arial;color:#082033;
  background: radial-gradient(800px 400px at 10% 10%, rgba(156,136,255,0.06), transparent 10%),
              radial-gradient(700px 350px at 90% 90%, rgba(0,198,255,0.06), transparent 10%),
              linear-gradient(180deg,var(--bg1),var(--bg2));
}

.blob{ position:fixed; border-radius:50%; filter: blur(36px); opacity:0.6; pointer-events:none; mix-blend-mode:screen; }
.blob.a{ width:380px; height:380px; left:-100px; top:-80px; background: linear-gradient(45deg,var(--accent3),var(--accent1)); }
.blob.b{ width:420px; height:420px; right:-140px; bottom:-120px; background: linear-gradient(45deg,var(--accent2),var(--accent3)); }

.wrap{ max-width:1100px; margin:34px auto; padding:18px; position:relative; }
.header{ display:flex; justify-content:space-between; margin-bottom:18px; }
.title{ font-weight:800; font-size:20px; }
.header-sub{ color:var(--muted); }

.grid{ display:grid; grid-template-columns: 1fr 380px; gap:18px; }

.card{ background:var(--card); border-radius:16px; padding:20px; }

.inputs{ display:grid; grid-template-columns: repeat(2,1fr); gap:12px; }
label{ font-weight:600; }
input[type=number]{ width:100%; padding:12px; border-radius:12px; border:1px solid #ccc; }

.btn{ padding:12px 16px; border-radius:12px; background: linear-gradient(90deg,var(--accent1),var(--accent2)); font-weight:800; border:none; }

.result{ margin-top:14px; padding:14px; border-radius:12px; font-weight:800; text-align:center; }
.result.good{ background:#d1fae5; color:#065f46; }
.result.bad{ background:#fee2e2; color:#7f1d1d; }

</style>
</head>
<body>

<div class="blob a"></div>
<div class="blob b"></div>

<div class="wrap">
  <div class="header">
    <div>
      <div class="title">BREAST CANCER DETECTION</div>
      <div class="header-sub">Accurate, fast, and designed for clarity</div>
    </div>
    <div style="text-align:right;">
      <div style="font-size:0.92rem; color:var(--muted)">Model Test Accuracy: <strong>{{ test_acc }}%</strong></div>
    </div>
  </div>

  <div class="grid">
    <div class="card">
      <div class="form-title">Enter patient feature values</div>

      <form method="POST" action="/predict">
        <div class="inputs">
          {% for f in features %}
          <div>
            <label>{{ f.replace('_',' ').title() }}</label>
            <input type="number" step="any" name="{{ f }}" required>
          </div>
          {% endfor %}
        </div>

        <button class="btn" type="submit">Predict</button>

        {% if result is defined %}
          <div class="result {{ 'good' if result=='Benign' else 'bad' }}">
            {{ result }}
          </div>
        {% endif %}
      </form>
    </div>

    <aside class="card">
      <div class="meta">About</div>
      <div class="hint">This demo uses a KNN model trained directly on your GitHub dataset.</div>

      <div style="margin-top:6px;" class="meta">Features used</div>
      <div class="hint">{{ features | join(', ') }}</div>
    </aside>

  </div>
</div>

</body>
</html>
"""


# =====================================================
# FLASK APP
# =====================================================

app = Flask(__name__)

@app.route("/", methods=["GET"])
def index():
    test_acc = round(knn.score(X_test, y_test) * 100, 2)
    return render_template_string(HTML, features=FEATURES, test_acc=test_acc)

@app.route("/predict", methods=["POST"])
def predict():
    values = [float(request.form.get(f)) for f in FEATURES]
    arr = np.array(values).reshape(1, -1)
    arr_scaled = scaler.transform(arr)
    pred = knn.predict(arr_scaled)[0]
    result = "Benign" if pred == 0 else "Malignant"
    test_acc = round(knn.score(X_test, y_test) * 100, 2)
    return render_template_string(HTML, features=FEATURES, result=result, test_acc=test_acc)


# RUN SERVER
def run():
    app.run(port=7860, debug=False, use_reloader=False)

threading.Thread(target=run).start()
time.sleep(1)
print("Launching app…")
output.serve_kernel_port_as_window(7860)


Dataset loaded successfully!

===== MODEL METRICS =====
Accuracy: 97.2 %
Precision: 94.55 %
Recall: 98.11 %
F1 Score: 96.3 %

Classification Report:

              precision    recall  f1-score   support

           0       0.99      0.97      0.98        90
           1       0.95      0.98      0.96        53

    accuracy                           0.97       143
   macro avg       0.97      0.97      0.97       143
weighted avg       0.97      0.97      0.97       143

 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 7860 is in use by another program. Either identify and stop that program, or start the server with a different port.


Launching app…
Try `serve_kernel_port_as_iframe` instead. 


<IPython.core.display.Javascript object>